In [2]:
import pandas as pd
import numpy as np
from plotly import express as px

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 1000)

In [35]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, roc_curve
import pandas as pd

In [3]:
import os
import glob
from os import listdir
from os.path import isfile, join
cur_dir = os. getcwd()
print(cur_dir)
mypath = os.path.join(cur_dir, "..","backend-project", "data")
files = [join(mypath, f) for f in listdir(mypath) if isfile(join(mypath, f))]

c:\Users\liule\Desktop\ETH\Spring2023\AIIML\Project\b5-winning-in-basketball\dev_notebooks


In [4]:
files

['c:\\Users\\liule\\Desktop\\ETH\\Spring2023\\AIIML\\Project\\b5-winning-in-basketball\\dev_notebooks\\..\\backend-project\\data\\dataset_games.csv',
 'c:\\Users\\liule\\Desktop\\ETH\\Spring2023\\AIIML\\Project\\b5-winning-in-basketball\\dev_notebooks\\..\\backend-project\\data\\dataset_games_details.csv',
 'c:\\Users\\liule\\Desktop\\ETH\\Spring2023\\AIIML\\Project\\b5-winning-in-basketball\\dev_notebooks\\..\\backend-project\\data\\dataset_players.csv',
 'c:\\Users\\liule\\Desktop\\ETH\\Spring2023\\AIIML\\Project\\b5-winning-in-basketball\\dev_notebooks\\..\\backend-project\\data\\dataset_ranking.csv',
 'c:\\Users\\liule\\Desktop\\ETH\\Spring2023\\AIIML\\Project\\b5-winning-in-basketball\\dev_notebooks\\..\\backend-project\\data\\dataset_teams.csv']

In [5]:
dfs = {}
for file_path in files:
    dfs[file_path.split(".")[-2].split("\\")[-1]] = pd.read_csv(file_path)

C:\Users\liule\AppData\Local\Temp\ipykernel_38188\871462510.py:3: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  dfs[file_path.split(".")[-2].split("\\")[-1]] = pd.read_csv(file_path)


In [6]:
dfs.keys()

dict_keys(['dataset_games', 'dataset_games_details', 'dataset_players', 'dataset_ranking', 'dataset_teams'])

In [7]:
dfs["dataset_games_details"].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 668628 entries, 0 to 668627
Data columns (total 29 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   GAME_ID            668628 non-null  int64  
 1   TEAM_ID            668628 non-null  int64  
 2   TEAM_ABBREVIATION  668628 non-null  object 
 3   TEAM_CITY          668628 non-null  object 
 4   PLAYER_ID          668628 non-null  int64  
 5   PLAYER_NAME        668628 non-null  object 
 6   NICKNAME           53037 non-null   object 
 7   START_POSITION     255765 non-null  object 
 8   COMMENT            109689 non-null  object 
 9   MIN                558938 non-null  object 
 10  FGM                558938 non-null  float64
 11  FGA                558938 non-null  float64
 12  FG_PCT             558938 non-null  float64
 13  FG3M               558938 non-null  float64
 14  FG3A               558938 non-null  float64
 15  FG3_PCT            558938 non-null  float64
 16  FT

In [8]:
# the features we use for our prediction
PRED_COLS = ['AST', 'BLK', 'DREB', 'FG3A', 'FG3M', 'FGA', 'FGM', 'FTA', 'FTM', 'OREB', 'PF', 'STL', 'TO']

REGULAR_SEASON_DURATION = {
    2021: ('2021-10-19', '2022-04-10'),
    2020: ('2020-12-22', '2021-05-16'),
    2018: ('2018-10-16', '2019-04-10'),
    2017: ('2017-10-17', '2018-04-11'),
}


In [9]:
def get_season_games(season=2021):
    games = dfs['dataset_games'].copy()
    games = games[['TEAM_ID_home', 'TEAM_ID_away', 'HOME_TEAM_WINS', 'GAME_ID', 'GAME_DATE_EST']]
    
    regular_games = games[(REGULAR_SEASON_DURATION[season][0] <= pd.to_datetime(games['GAME_DATE_EST'])) & (pd.to_datetime(games['GAME_DATE_EST']) <= REGULAR_SEASON_DURATION[season][1])]
    
    return regular_games

def get_team_boxscore(team_id=1610612738, is_home=True):
    games = get_season_games(2021)
    games_details = dfs['dataset_games_details'].copy()
    # join games to games_details for date information
    games_details = games_details.merge(games, on='GAME_ID')

    # add additional column indicating whether team is home or not
    games_details['is_home'] = games_details['TEAM_ID'] == games_details['TEAM_ID_home']

    # select data from team at home or away
    games_details = games_details[(games_details['TEAM_ID']==team_id) & (games_details['is_home']==is_home)][PRED_COLS+['GAME_ID']]

    # sum over all players for each game and then average over all games
    boxscore = games_details.groupby(['GAME_ID']).sum().mean().to_frame().T
    
    return boxscore

def get_team_boxscore_by_position(team_id=1610612738, is_home=True, start_position = 'F'):
    games = get_season_games(2021)
    games_details = dfs['dataset_games_details'].copy()
    # join games to games_details for date information
    games_details = games_details.merge(games, on='GAME_ID')

    # add additional column indicating whether team is home or not
    games_details['is_home'] = games_details['TEAM_ID'] == games_details['TEAM_ID_home']

    # select data from team at home or away
    games_details = games_details[(games_details['TEAM_ID']==team_id) & (games_details['START_POSITION']==start_position) & (games_details['is_home']==is_home)][PRED_COLS+['GAME_ID', 'START_POSITION']]

    # sum over all players for each game and then average over all games
    boxscore = games_details.groupby(['GAME_ID']).sum().mean().to_frame().T
    
    return boxscore

In [10]:
get_season_games()

,TEAM_ID_home,TEAM_ID_away,HOME_TEAM_WINS,GAME_ID,GAME_DATE_EST
635,1610612751,1610612754,1,22101216,2022-04-10
636,1610612766,1610612764,1,22101217,2022-04-10
637,1610612739,1610612749,1,22101218,2022-04-10
638,1610612745,1610612737,0,22101221,2022-04-10
639,1610612763,1610612738,0,22101223,2022-04-10
...,...,...,...,...,...
1860,1610612762,1610612760,1,22100011,2021-10-20
1861,1610612756,1610612743,0,22100012,2021-10-20
1862,1610612757,1610612758,0,22100013,2021-10-20
1863,1610612749,1610612751,1,22100001,2021-10-19


In [11]:
box_score = get_team_boxscore()
box_score

,AST,BLK,DREB,FG3A,FG3M,FGA,FGM,FTA,FTM,OREB,PF,STL,TO
0,24.512195,5.682927,34.707317,35.682927,12.560976,87.268293,40.707317,20.512195,16.780488,10.902439,18.0,7.463415,13.097561


In [29]:
F_box_score = get_team_boxscore_by_position(start_position='C')
F_box_score

,AST,BLK,DREB,FG3A,FG3M,FGA,FGM,FTA,FTM,OREB,PF,STL,TO
0,2.487805,1.95122,5.97561,0.731707,0.243902,7.097561,4.853659,1.585366,1.121951,3.658537,2.243902,1.0,1.268293


In [13]:
def get_season_boxscore(season=2021):
    games = get_season_games(season)
    games_details = dfs['dataset_games_details'].copy()
    
    # add games details to regular season games
    games_details = games_details.merge(games, on='GAME_ID')

    # add additional column indicating whether team is home or not
    games_details['is_home'] = games_details['TEAM_ID'] == games_details['TEAM_ID_home']
    
    games_details = games_details[PRED_COLS + ['GAME_ID', 'TEAM_ID', 'is_home']]
    # group by game and team and sum stats
    games_details_agg = games_details.groupby(['GAME_ID', 'TEAM_ID']).sum()
    # reset is_home col to boolean value
    games_details_agg['is_home'] = games_details_agg['is_home'] > 0
    # group by team and is_home
    games_details_agg = games_details_agg.groupby(['TEAM_ID', 'is_home']).mean()
    # only keep prediction columns
    games_details_agg = games_details_agg[PRED_COLS]
    games_details_agg = games_details_agg.reset_index()
    return games_details_agg


def get_train_data(season=2021):
    season_games = get_season_games(season)
    games_details = get_season_boxscore(season)
    
    # add stats of home teams
    train_data = season_games.merge(games_details[games_details['is_home']].drop(columns='is_home'), left_on='TEAM_ID_home', right_on='TEAM_ID')
    # rename stats cols
    train_data = train_data.rename(columns={col: f"{col}_home" for col in PRED_COLS})
    # add stats of away teams
    train_data = train_data.merge(games_details[~games_details['is_home']].drop(columns='is_home'), left_on='TEAM_ID_away', right_on='TEAM_ID')
    # rename stats cols
    train_data = train_data.rename(columns={col: f"{col}_away" for col in PRED_COLS})
    
    # only return feature and target columns
    return train_data[[f"{col}_home" for col in PRED_COLS]+[f"{col}_away" for col in PRED_COLS]+['HOME_TEAM_WINS']]

In [14]:
get_season_boxscore()

,TEAM_ID,is_home,AST,BLK,DREB,FG3A,FG3M,FGA,FGM,FTA,FTM,OREB,PF,STL,TO
0,1610612737,False,24.000000,3.878049,33.609756,33.634146,12.414634,89.585366,41.048780,20.829268,16.707317,10.731707,18.487805,6.512195,11.439024
1,1610612737,True,25.195122,4.609756,34.268293,35.170732,13.341463,87.024390,41.902439,23.780488,19.512195,9.341463,18.926829,7.804878,11.097561
2,1610612738,False,25.146341,5.975610,36.390244,38.560976,13.902439,87.536585,40.780488,21.268293,17.292683,10.146341,19.097561,6.951220,13.000000
3,1610612738,True,24.512195,5.682927,34.707317,35.682927,12.560976,87.268293,40.707317,20.512195,16.780488,10.902439,18.000000,7.463415,13.097561
4,1610612739,False,24.926829,4.121951,33.390244,31.658537,11.048780,84.560976,39.756098,21.902439,16.951220,10.195122,17.439024,6.853659,13.341463
5,1610612739,True,25.414634,4.243902,34.560976,33.853659,12.195122,84.707317,39.634146,22.243902,16.609756,10.219512,17.512195,7.292683,13.195122
6,1610612740,False,24.073171,4.000000,33.292683,30.682927,10.219512,87.024390,38.975610,24.414634,19.048780,11.780488,19.536585,8.512195,13.365854
7,1610612740,True,25.853659,4.000000,33.195122,33.439024,11.073171,88.878049,41.365854,22.000000,17.560976,12.146341,19.780488,8.073171,13.195122
8,1610612741,False,23.463415,4.121951,32.975610,29.097561,10.658537,85.658537,40.682927,20.829268,17.000000,8.024390,19.219512,7.317073,12.951220
9,1610612741,True,24.292683,4.073171,34.341463,28.560976,10.609756,88.170732,42.780488,22.243902,18.024390,9.317073,18.341463,6.951220,11.292683


In [16]:
get_train_data()

,AST_home,BLK_home,DREB_home,FG3A_home,FG3M_home,FGA_home,FGM_home,FTA_home,FTM_home,OREB_home,PF_home,STL_home,TO_home,AST_away,BLK_away,DREB_away,FG3A_away,FG3M_away,FGA_away,FGM_away,FTA_away,FTM_away,OREB_away,PF_away,STL_away,TO_away,HOME_TEAM_WINS
0,25.073171,5.707317,34.536585,31.121951,10.317073,88.439024,41.195122,21.926829,17.731707,10.975610,19.682927,6.951220,14.365854,23.536585,5.609756,32.926829,35.268293,12.000000,88.682927,39.878049,22.292683,16.951220,11.000000,20.731707,6.804878,13.731707,1
1,25.073171,5.707317,34.536585,31.121951,10.317073,88.439024,41.195122,21.926829,17.731707,10.975610,19.682927,6.951220,14.365854,23.536585,5.609756,32.926829,35.268293,12.000000,88.682927,39.878049,22.292683,16.951220,11.000000,20.731707,6.804878,13.731707,1
2,28.268293,5.560976,34.951220,38.707317,13.560976,91.317073,42.390244,20.926829,15.121951,11.390244,20.048780,7.926829,13.341463,23.536585,5.609756,32.926829,35.268293,12.000000,88.682927,39.878049,22.292683,16.951220,11.000000,20.731707,6.804878,13.731707,1
3,28.268293,5.560976,34.951220,38.707317,13.560976,91.317073,42.390244,20.926829,15.121951,11.390244,20.048780,7.926829,13.341463,23.536585,5.609756,32.926829,35.268293,12.000000,88.682927,39.878049,22.292683,16.951220,11.000000,20.731707,6.804878,13.731707,1
4,25.414634,4.243902,34.560976,33.853659,12.195122,84.707317,39.634146,22.243902,16.609756,10.219512,17.512195,7.292683,13.195122,23.536585,5.609756,32.926829,35.268293,12.000000,88.682927,39.878049,22.292683,16.951220,11.000000,20.731707,6.804878,13.731707,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1225,27.219512,5.536585,32.219512,35.439024,12.317073,90.292683,43.000000,20.487805,15.902439,11.560976,20.121951,7.390244,13.512195,25.439024,5.219512,33.658537,32.341463,12.609756,88.414634,42.756098,21.512195,17.243902,9.609756,21.048780,7.243902,12.121951,0
1226,22.634146,5.000000,35.170732,37.341463,12.341463,88.829268,38.926829,18.975610,14.463415,10.609756,18.609756,7.658537,13.317073,25.439024,5.219512,33.658537,32.341463,12.609756,88.414634,42.756098,21.512195,17.243902,9.609756,21.048780,7.243902,12.121951,0
1227,23.756098,4.658537,33.195122,32.634146,11.658537,86.195122,40.048780,23.780488,18.560976,8.975610,19.195122,7.292683,13.756098,25.439024,5.219512,33.658537,32.341463,12.609756,88.414634,42.756098,21.512195,17.243902,9.609756,21.048780,7.243902,12.121951,1
1228,24.512195,5.682927,34.707317,35.682927,12.560976,87.268293,40.707317,20.512195,16.780488,10.902439,18.000000,7.463415,13.097561,25.439024,5.219512,33.658537,32.341463,12.609756,88.414634,42.756098,21.512195,17.243902,9.609756,21.048780,7.243902,12.121951,1


In [209]:
def calculate_advanced_stat(train_data):
    train_data_advanced = train_data.copy()
    for index, row in train_data_advanced.iterrows():
        
        # points
        points_home = 2 * (row["FGM_home"] + 0.5 * row["FG3M_home"]) + row["FTM_home"]
        points_away = 2 * (row["FGM_away"] + 0.5 * row["FG3M_away"]) + row["FTM_away"]
        train_data_advanced.loc[index, "Points_home"] = points_home
        train_data_advanced.loc[index, "Points_away"] = points_away

        # possession = .5 * (FGA + .475 * FTA - ORB + TOV)
        train_data_advanced.loc[index, "Possesions_home"] = 0.5 * (row["FGA_home"] + 0.475 * row["FTA_home"] - row["OREB_home"] + row["TO_home"])
        train_data_advanced.loc[index, "Possesions_away"] = 0.5 * (row["FGA_away"] + 0.475 * row["FTA_away"] - row["OREB_away"] + row["TO_away"])

        # Effective Field Goal Percentage (eFG%) = (FG + .5 * 3P) / FGA
        train_data_advanced.loc[index, "EFG%_home"] = (row["FGM_home"] + 0.5 * row["FG3M_home"]) / row["FGA_home"]
        train_data_advanced.loc[index, "EFG%_away"] = (row["FGM_away"] + 0.5 * row["FG3M_away"]) / row["FGA_away"]

        # True Shooting Percentage (TS%) = Pts / (2 * (FGA + .475 * FTA))
        train_data_advanced.loc[index, "TS%_home"] = points_home / (2 * (row["FGA_home"] + 0.475 * row["FTA_home"]))
        train_data_advanced.loc[index, "TS%_away"] = points_away / (2 * (row["FGA_away"] + 0.475 * row["FTA_away"]))

        # Turnover Percentage (TOV%) = TOV / (FGA + .475*FTA + AST + TOV)
        train_data_advanced.loc[index, "TOV%_home"] = row["TO_home"] / (row["FGA_home"] + 0.475 * row["FTA_home"] + row["TO_home"])
        train_data_advanced.loc[index, "TOV%_away"] = row["TO_away"] / (row["FGA_away"] + 0.475 * row["FTA_away"] + row["TO_away"])

    return train_data_advanced

In [210]:
train_data = []
for season in REGULAR_SEASON_DURATION:
    train_data.append(get_train_data(season))
    
train_data = pd.concat(train_data, axis=0)

In [211]:
train_data_advanced = calculate_advanced_stat(train_data)

In [212]:
train_data_advanced

,AST_home,BLK_home,DREB_home,FG3A_home,FG3M_home,FGA_home,FGM_home,FTA_home,FTM_home,OREB_home,PF_home,STL_home,TO_home,AST_away,BLK_away,DREB_away,FG3A_away,FG3M_away,FGA_away,FGM_away,FTA_away,FTM_away,OREB_away,PF_away,STL_away,TO_away,HOME_TEAM_WINS,Points_home,Points_away,Possesions_home,Possesions_away,EFG%_home,EFG%_away,TS%_home,TS%_away,TOV%_home,TOV%_away
0,25.073171,5.707317,34.536585,31.121951,10.317073,88.439024,41.195122,21.926829,17.731707,10.975610,19.682927,6.951220,14.365854,23.536585,5.609756,32.926829,35.268293,12.000000,88.682927,39.878049,22.292683,16.951220,11.000000,20.731707,6.804878,13.731707,1,105.317073,107.341463,49.608537,51.425305,0.520703,0.518633,0.549798,0.550280,0.117531,0.132918
1,25.073171,5.707317,34.536585,31.121951,10.317073,88.439024,41.195122,21.926829,17.731707,10.975610,19.682927,6.951220,14.365854,23.536585,5.609756,32.926829,35.268293,12.000000,88.682927,39.878049,22.292683,16.951220,11.000000,20.731707,6.804878,13.731707,1,105.317073,107.341463,49.608537,51.425305,0.520703,0.518633,0.549798,0.550280,0.117531,0.132918
2,28.268293,5.560976,34.951220,38.707317,13.560976,91.317073,42.390244,20.926829,15.121951,11.390244,20.048780,7.926829,13.341463,23.536585,5.609756,32.926829,35.268293,12.000000,88.682927,39.878049,22.292683,16.951220,11.000000,20.731707,6.804878,13.731707,1,111.463415,107.341463,50.546341,51.425305,0.549383,0.518633,0.579537,0.550280,0.121049,0.132918
3,28.268293,5.560976,34.951220,38.707317,13.560976,91.317073,42.390244,20.926829,15.121951,11.390244,20.048780,7.926829,13.341463,23.536585,5.609756,32.926829,35.268293,12.000000,88.682927,39.878049,22.292683,16.951220,11.000000,20.731707,6.804878,13.731707,1,111.463415,107.341463,50.546341,51.425305,0.549383,0.518633,0.579537,0.550280,0.121049,0.132918
4,25.414634,4.243902,34.560976,33.853659,12.195122,84.707317,39.634146,22.243902,16.609756,10.219512,17.512195,7.292683,13.195122,23.536585,5.609756,32.926829,35.268293,12.000000,88.682927,39.878049,22.292683,16.951220,11.000000,20.731707,6.804878,13.731707,1,104.634146,107.341463,49.274085,51.425305,0.523911,0.518633,0.550922,0.550280,0.123580,0.132918
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1225,24.560976,6.463415,34.731707,33.000000,12.024390,88.926829,41.804878,20.926829,16.536585,10.121951,20.902439,7.439024,12.365854,20.829268,3.853659,34.390244,29.975610,11.536585,84.121951,37.512195,20.804878,16.146341,9.390244,19.146341,7.634146,13.853659,1,112.170732,102.707317,50.555488,49.233841,0.537713,0.514497,0.567281,0.546291,0.111171,0.128444
1226,24.560976,6.463415,34.731707,33.000000,12.024390,88.926829,41.804878,20.926829,16.536585,10.121951,20.902439,7.439024,12.365854,20.829268,3.853659,34.390244,29.975610,11.536585,84.121951,37.512195,20.804878,16.146341,9.390244,19.146341,7.634146,13.853659,1,112.170732,102.707317,50.555488,49.233841,0.537713,0.514497,0.567281,0.546291,0.111171,0.128444
1227,19.951220,5.121951,33.926829,27.951220,9.000000,87.585366,38.414634,23.853659,17.341463,10.341463,22.463415,6.512195,15.097561,20.829268,3.853659,34.390244,29.975610,11.536585,84.121951,37.512195,20.804878,16.146341,9.390244,19.146341,7.634146,13.853659,0,103.170732,102.707317,51.835976,49.233841,0.489975,0.514497,0.521508,0.546291,0.132419,0.128444
1228,22.414634,4.707317,35.073171,43.317073,15.463415,84.731707,39.219512,26.658537,20.487805,9.292683,19.390244,8.926829,13.292683,20.829268,3.853659,34.390244,29.975610,11.536585,84.121951,37.512195,20.804878,16.146341,9.390244,19.146341,7.634146,13.853659,1,114.390244,102.707317,50.697256,49.233841,0.554116,0.514497,0.587252,0.546291,0.120092,0.128444


In [213]:
PREDICTION_STATS = ['AST_home', 'BLK_home', 'DREB_home', 'OREB_home', 'PF_home', 'STL_home', 'TO_home', "TS%_home", "EFG%_home", "TOV%_home", "Possesions_home",
                    'AST_away', 'BLK_away', 'DREB_away', 'OREB_away', 'PF_away', 'STL_away', 'TO_away', "TS%_away", "EFG%_away", "TOV%_away", "Possesions_away"] 

In [214]:
y = train_data_advanced["HOME_TEAM_WINS"]
X = train_data_advanced[PREDICTION_STATS]

In [215]:
X

,AST_home,BLK_home,DREB_home,OREB_home,PF_home,STL_home,TO_home,TS%_home,EFG%_home,TOV%_home,Possesions_home,AST_away,BLK_away,DREB_away,OREB_away,PF_away,STL_away,TO_away,TS%_away,EFG%_away,TOV%_away,Possesions_away
0,25.073171,5.707317,34.536585,10.975610,19.682927,6.951220,14.365854,0.549798,0.520703,0.117531,49.608537,23.536585,5.609756,32.926829,11.000000,20.731707,6.804878,13.731707,0.550280,0.518633,0.132918,51.425305
1,25.073171,5.707317,34.536585,10.975610,19.682927,6.951220,14.365854,0.549798,0.520703,0.117531,49.608537,23.536585,5.609756,32.926829,11.000000,20.731707,6.804878,13.731707,0.550280,0.518633,0.132918,51.425305
2,28.268293,5.560976,34.951220,11.390244,20.048780,7.926829,13.341463,0.579537,0.549383,0.121049,50.546341,23.536585,5.609756,32.926829,11.000000,20.731707,6.804878,13.731707,0.550280,0.518633,0.132918,51.425305
3,28.268293,5.560976,34.951220,11.390244,20.048780,7.926829,13.341463,0.579537,0.549383,0.121049,50.546341,23.536585,5.609756,32.926829,11.000000,20.731707,6.804878,13.731707,0.550280,0.518633,0.132918,51.425305
4,25.414634,4.243902,34.560976,10.219512,17.512195,7.292683,13.195122,0.550922,0.523911,0.123580,49.274085,23.536585,5.609756,32.926829,11.000000,20.731707,6.804878,13.731707,0.550280,0.518633,0.132918,51.425305
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1225,24.560976,6.463415,34.731707,10.121951,20.902439,7.439024,12.365854,0.567281,0.537713,0.111171,50.555488,20.829268,3.853659,34.390244,9.390244,19.146341,7.634146,13.853659,0.546291,0.514497,0.128444,49.233841
1226,24.560976,6.463415,34.731707,10.121951,20.902439,7.439024,12.365854,0.567281,0.537713,0.111171,50.555488,20.829268,3.853659,34.390244,9.390244,19.146341,7.634146,13.853659,0.546291,0.514497,0.128444,49.233841
1227,19.951220,5.121951,33.926829,10.341463,22.463415,6.512195,15.097561,0.521508,0.489975,0.132419,51.835976,20.829268,3.853659,34.390244,9.390244,19.146341,7.634146,13.853659,0.546291,0.514497,0.128444,49.233841
1228,22.414634,4.707317,35.073171,9.292683,19.390244,8.926829,13.292683,0.587252,0.554116,0.120092,50.697256,20.829268,3.853659,34.390244,9.390244,19.146341,7.634146,13.853659,0.546291,0.514497,0.128444,49.233841


### Classification

In [216]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [116]:
from sklearn.model_selection import cross_val_score
import statistics

In [121]:
from sklearn.model_selection import train_test_split
from interpret.glassbox import ExplainableBoostingClassifier
from interpret import show
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.kernel_ridge import KernelRidge
from sklearn.gaussian_process.kernels import RBF

In [198]:
ebm = ExplainableBoostingClassifier(learning_rate= 0.003, max_leaves=64, max_rounds= 10000, feature_names = PREDICTION_STATS, early_stopping_rounds= 100,
                                   interactions=5,validation_size=0.3,early_stopping_tolerance=0.0001,inner_bags=32,outer_bags=32,binning="rounded_quantile",max_bins=512
                                   )

scores_ebm_ac = cross_val_score(ebm, X, y, cv=5, scoring="accuracy")
scores_ebm_f1 = cross_val_score(ebm, X, y, cv=5, scoring="f1")

In [199]:
print("accuracy of 5-fold CV:")
print(statistics.mean(scores_ebm_ac))

print("F1 of 5-fold CV:")
print(statistics.mean(scores_ebm_f1))

accuracy of 5-fold CV:
0.5478321602363574
F1 of 5-fold CV:
0.6186367567186924


In [254]:
# using sklearn API

# Define the LightGBM model
model = lgb.LGBMClassifier(boosting_type='dart', num_leaves=1023, max_depth=10, learning_rate=0.01, n_estimators=50, min_child_samples=500, random_state=0, objective='binary')

model_ac = cross_val_score(model, X, y, cv=5, scoring="accuracy")
model_f1 = cross_val_score(model, X, y, cv=5, scoring="f1")

In [256]:
print("accuracy of 5-fold CV:")
print(statistics.mean(model_ac))

print("F1 of 5-fold CV:")
print(statistics.mean(model_f1))


accuracy of 5-fold CV:
0.5655344108446299
F1 of 5-fold CV:
0.7224809541476673


#### Clustering